In [ ]:
import pandas as pd
import numpy as np

# Company symbols and file names
companies = ["AMZN", "GOOG", "MSFT", "TSLA", "NVDA"]

# RSI calculation function
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Process each file
for company in companies:
    input_file = f"{company}_2025_data.xlsx"
    output_file = f"{company}_2025_enriched.xlsx"

    try:
        df = pd.read_excel(input_file)

        # Sort by date
        df.sort_values("Date", inplace=True)
        df.reset_index(drop=True, inplace=True)

        # Calculate log return
        df["Return"] = np.log(df["Close"] / df["Close"].shift(1))

        # Volatility: 14-day moving standard deviation
        df["Volatility"] = df["Return"].rolling(window=14).std()

        # RSI
        df["RSI"] = compute_rsi(df["Close"], period=14)

        # Value: (Close - Open) / Open
        df["Value"] = (df["Close"] - df["Open"]) / df["Open"]

        # Save to new file
        df.to_excel(output_file, index=False)
        print(f"{output_file} successfully created.")
        
    except FileNotFoundError:
        print(f"{input_file} not found, skipping.")